In [ ]:
!pip install ffmpeg opencv-python opencv-python-headless basicsr facexlib gfpgan tqdm -q
!pip install -U git+https://github.com/xinntao/Real-ESRGAN.git -q



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!wget -P experiments/pretrained_models https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
!pip install -r requirements.txt
!python setup.py develop
%cd ..

file_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"
with open(file_path, "r") as f:
    lines = f.readlines()
with open(file_path, "w") as f:
    for line in lines:
        if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
            f.write('from torchvision.transforms.functional import rgb_to_grayscale\n')
        else:
            f.write(line)



Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 10.80 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
--2025-06-26 13:38:37--  https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/387326890/08f0e941-ebb7-48f0-9d6a-73e87b710e7e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250626%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250626T133837Z&X-Amz-Expires=1800&X-Amz-Signature=3f4ba5af918a3a7ca6b8344fcba5aeacd4b

In [ ]:
from google.colab import files
print("Upload your video")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]



Upload your video


Saving harry.mp4 to harry.mp4


In [ ]:
import os
os.makedirs('frames', exist_ok=True)
!ffmpeg -i "$video_path" -qscale:v 1 frames/frame%04d.png



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from glob import glob
import subprocess
import os

input_dir = 'frames'
output_dir = 'frames_upscaled'
os.makedirs(output_dir, exist_ok=True)

frame_list = sorted(glob(os.path.join(input_dir, '*.png')))
total = len(frame_list)

print(f"Upscaling {total} frames...")

# Run ESRGAN as subprocess and print output line-by-line
process = subprocess.Popen(
    ['python', 'Real-ESRGAN/inference_realesrgan.py',
     '-n', 'RealESRGAN_x4plus',
     '-i', input_dir,
     '-o', output_dir],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Print Real-ESRGAN output directly
for line in process.stdout:
    print(line.strip())

process.wait()


Upscaling 85 frames...
Testing 0 frame0001
Testing 1 frame0002
Testing 2 frame0003
Testing 3 frame0004
Testing 4 frame0005
Testing 5 frame0006
Testing 6 frame0007
Testing 7 frame0008
Testing 8 frame0009
Testing 9 frame0010
Testing 10 frame0011
Testing 11 frame0012
Testing 12 frame0013
Testing 13 frame0014
Testing 14 frame0015
Testing 15 frame0016
Testing 16 frame0017
Testing 17 frame0018
Testing 18 frame0019
Testing 19 frame0020
Testing 20 frame0021
Testing 21 frame0022
Testing 22 frame0023
Testing 23 frame0024
Testing 24 frame0025
Testing 25 frame0026
Testing 26 frame0027
Testing 27 frame0028
Testing 28 frame0029
Testing 29 frame0030
Testing 30 frame0031
Testing 31 frame0032
Testing 32 frame0033
Testing 33 frame0034
Testing 34 frame0035
Testing 35 frame0036
Testing 36 frame0037
Testing 37 frame0038
Testing 38 frame0039
Testing 39 frame0040
Testing 40 frame0041
Testing 41 frame0042
Testing 42 frame0043
Testing 43 frame0044
Testing 44 frame0045
Testing 45 frame0046
Testing 46 frame0047


0

In [ ]:
output_video = "upscaled_output_4k.mp4"
!ffmpeg -y -r 29.97 -start_number 1 -i frames_upscaled/frame%04d_out.png -vf "scale=3840:2160" -c:v libx264 -crf 16 -preset slow -pix_fmt yuv420p "$output_video"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab